# Railway Image Sample Selector

Dieses Notebook wählt 10 zufällige Bilder aus dem `datasets` Ordner aus und kopiert sie in den `datasets/clustering_sample_100` Ordner, der vom `img_preprocessing` Notebook benötigt wird.

## Funktionen:
- **Zufällige Auswahl**: Wählt 10 zufällige Bilder aus dem verfügbaren Dataset
- **Automatisches Kopieren**: Kopiert die ausgewählten Bilder in den Zielordner
- **Ordner-Management**: Erstellt den Zielordner falls er nicht existiert
- **Vorschau**: Zeigt die ausgewählten Bilder zur Kontrolle an

## Verwendung:
1. Führen Sie alle Zellen der Reihe nach aus
2. Die 10 ausgewählten Bilder werden angezeigt
3. Die Bilder werden automatisch in `datasets/clustering_sample_100` kopiert
4. Danach können Sie das `img_preprocessing` Notebook verwenden

In [ ]:
# Import Required Libraries
import os
import shutil
import random
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import time

# Configuration
SOURCE_FOLDER = "./datasets/clustering_sample_10000"  # Quellordner mit vielen Bildern
TARGET_FOLDER = "./datasets/clustering_sample_100"    # Zielordner für img_preprocessing
SAMPLE_SIZE = 10                                      # Anzahl der auszuwählenden Bilder
RANDOM_SEED = 42                                      # Für reproduzierbare Ergebnisse

# Supported image formats
SUPPORTED_FORMATS = ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']

print("🚂 Railway Image Sample Selector")
print("=" * 40)
print(f"Quellordner: {SOURCE_FOLDER}")
print(f"Zielordner: {TARGET_FOLDER}")
print(f"Anzahl Bilder: {SAMPLE_SIZE}")
print(f"Random Seed: {RANDOM_SEED}")

In [ ]:
# Check source folder and find available images
source_path = Path(SOURCE_FOLDER)

if not source_path.exists():
    print(f"❌ Fehler: Quellordner {SOURCE_FOLDER} existiert nicht!")
    print("\n📁 Verfügbare Ordner im datasets Verzeichnis:")
    datasets_dir = Path("./datasets")
    if datasets_dir.exists():
        for subdir in datasets_dir.iterdir():
            if subdir.is_dir():
                img_count = len([f for f in subdir.iterdir() if f.suffix in SUPPORTED_FORMATS])
                print(f"  📂 {subdir.name} ({img_count} Bilder)")
    else:
        print("  ❌ datasets Ordner existiert nicht!")
else:
    # Find all image files in source folder
    image_files = [f for f in source_path.iterdir() if f.suffix in SUPPORTED_FORMATS]
    
    print(f"✅ Quellordner gefunden: {SOURCE_FOLDER}")
    print(f"📊 Verfügbare Bilder: {len(image_files)}")
    
    if len(image_files) < SAMPLE_SIZE:
        print(f"⚠️  Warnung: Nur {len(image_files)} Bilder verfügbar, aber {SAMPLE_SIZE} angefordert!")
        print(f"📝 Auswahl wird auf {len(image_files)} Bilder reduziert.")
        SAMPLE_SIZE = len(image_files)
    
    if len(image_files) == 0:
        print("❌ Keine Bilder im Quellordner gefunden!")
    else:
        print(f"🎯 {SAMPLE_SIZE} Bilder werden ausgewählt.")

In [ ]:
# Select random images
if len(image_files) > 0:
    # Set random seed for reproducibility
    random.seed(RANDOM_SEED)
    
    # Randomly sample images
    selected_images = random.sample(image_files, SAMPLE_SIZE)
    
    print(f"🎲 {SAMPLE_SIZE} zufällige Bilder ausgewählt:")
    print("=" * 50)
    
    for i, img_path in enumerate(selected_images, 1):
        # Get file size
        file_size = img_path.stat().st_size / 1024  # KB
        print(f"{i:2d}. {img_path.name} ({file_size:.1f} KB)")
    
    print(f"\n📋 Gesamtauswahl: {len(selected_images)} Bilder")
else:
    print("❌ Keine Bilder zum Auswählen verfügbar!")
    selected_images = []

In [ ]:
# Create target folder and copy images
if selected_images:
    target_path = Path(TARGET_FOLDER)
    
    # Create target folder if it doesn't exist
    if not target_path.exists():
        target_path.mkdir(parents=True, exist_ok=True)
        print(f"📁 Zielordner erstellt: {TARGET_FOLDER}")
    else:
        # Clean existing files in target folder
        existing_files = list(target_path.iterdir())
        if existing_files:
            print(f"🧹 {len(existing_files)} existierende Dateien im Zielordner werden gelöscht...")
            for file in existing_files:
                if file.is_file():
                    file.unlink()
    
    # Copy selected images
    print(f"\n📂 Kopiere {len(selected_images)} Bilder nach {TARGET_FOLDER}...")
    print("=" * 60)
    
    copied_files = []
    failed_copies = []
    
    for i, source_file in enumerate(selected_images, 1):
        try:
            target_file = target_path / source_file.name
            shutil.copy2(source_file, target_file)
            copied_files.append(target_file)
            print(f"✅ {i:2d}/#{len(selected_images)} - {source_file.name}")
        except Exception as e:
            failed_copies.append((source_file.name, str(e)))
            print(f"❌ {i:2d}/#{len(selected_images)} - {source_file.name} (Fehler: {e})")
    
    # Summary
    print(f"\n📊 Kopiervorgang abgeschlossen:")
    print(f"   ✅ Erfolgreich kopiert: {len(copied_files)}")
    print(f"   ❌ Fehlgeschlagen: {len(failed_copies)}")
    
    if failed_copies:
        print("\n❌ Fehlerhafte Dateien:")
        for filename, error in failed_copies:
            print(f"   - {filename}: {error}")
    
    if copied_files:
        print(f"\n🎉 Erfolgreich! {len(copied_files)} Bilder sind bereit für das img_preprocessing Notebook.")
        print(f"📁 Zielordner: {TARGET_FOLDER}")
else:
    print("❌ Keine Bilder zum Kopieren verfügbar!")

In [ ]:
# Display preview of selected images
if copied_files:
    print("🖼️  Vorschau der ausgewählten Bilder:")
    print("=" * 40)
    
    # Calculate grid layout
    n_images = len(copied_files)
    cols = min(5, n_images)  # Max 5 columns
    rows = (n_images + cols - 1) // cols  # Calculate needed rows
    
    # Create figure
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    
    # Handle single row case
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()
    
    # Display images
    for i, img_path in enumerate(copied_files):
        try:
            # Load and display image
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].set_title(f"{i+1}. {img_path.name}\n{img.size[0]}x{img.size[1]}", 
                            fontsize=8, pad=5)
            axes[i].axis('off')
        except Exception as e:
            axes[i].text(0.5, 0.5, f'Fehler beim\nLaden von\n{img_path.name}', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f"{i+1}. {img_path.name} (Fehler)", fontsize=8)
            axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(copied_files), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f'Ausgewählte Bilder für Image Preprocessing ({len(copied_files)} Bilder)', 
                 fontsize=14, y=1.02)
    plt.show()
    
    # Final instructions
    print(f"\n🚀 Nächste Schritte:")
    print(f"   1. Öffnen Sie das 'img_preprocessing.ipynb' Notebook")
    print(f"   2. Führen Sie alle Zellen aus")
    print(f"   3. Die {len(copied_files)} Bilder werden automatisch verarbeitet")
    print(f"\n📂 Ihr Sample-Dataset ist bereit: {TARGET_FOLDER}")
else:
    print("❌ Keine Bilder für die Vorschau verfügbar!")